# 05-supervisor.ipynb
멀티 에이전트 패턴 중 하나
- 도메인이 여러개 섞여 있고
- 각 도메인마다 도구가 많고 복잡함
- 하위 담당 에이전트와 사용자가 소통 할 필요가 없음
- 단순 도구만 활용할 경우에는 사용 X

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model('gpt-4.1-mini')

In [ ]:
from langchain.tools import tool

@tool
def create_calendar_event(
    title: str,
    start_time: str,
    end_time: str,
    attendees: list[str],  # ['a@a.com', 'b@b.com',]
    location: str = '',    # 위치가 없을 경우, 빈 문자열
):
    '''캘린더 이벤트 생성'''
    
    return f'이벤트 생성 완료. {title} - {start_time} ~ {end_time}. '


@tool
def send_email(
    to: list[str],
    subject: str,
    body: str,
    attendees: list[str],
):
    '''이메일 발송'''

    return f'이메일 발송 완료. {to} - {subject}'


@tool
def get_available_time_slot(
    attendees: list[str],
    date: str,
    duration_minutes: int
):
    '''참가자들이 특정 날짜에 참여 가능한 시간 확인'''

    return ['09:00', '14:00', '16:00']

In [ ]:
from langchain.agents import create_agent
from datetime import datetime

CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response."
    f"NOW: {datetime.now()}"
)

calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slot],
    system_prompt=CALENDAR_AGENT_PROMPT
)

In [ ]:
query = '다음주 화요일 오전 10시에 1시간동안 팀 미팅을 잡아줘'

for step in calendar_agent.stream(
    {'messages': [{'role': 'user', 'content': query}]}
):
    for update in step.values():
        for message in update.get('messages', []):
            message.pretty_print()

In [ ]:
EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response."
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
)

In [ ]:
query = '디자인 팀한테 내일 오전 10시에 있는 디자인 리뷰 리마인더 보내줘'

for step in email_agent.stream(
    {'messages': [{'role': 'user', 'content': query}]}
):
    for update in step.values():
        for message in update.get('messages', []):
            message.pretty_print()

In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit

import os

DB_URI = os.environ.get('DB_URI')

db = SQLDatabase.from_uri(DB_URI)
toolkit = SQLDatabaseToolkit(db=db, llm=model)

dialect = db.dialect
top_k = 5

SQL_AGENT_PROMPT = f"""
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
"""

db_agent = create_agent(
    model, 
    toolkit.get_tools(), 
    system_prompt=SQL_AGENT_PROMPT,
)

In [ ]:
query = '백엔드 팀에 있는 사람들의 이메일주소만 알려줘'

for step in db_agent.stream(
    {'messages': [{'role': 'user', 'content': query}]}
):
    for update in step.values():
        for message in update.get('messages', []):
            message.pretty_print()

In [ ]:
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


@tool
def query_db(request: str) -> str:
    """Query Database using natural language.

    Team info and employee info are in Database.
    Use this when you need to query DB.

    Input: Natural language query request (e.g., 'members in data team.')
    """
    result = db_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

SUPERVISOR_PROMPT = '''너는 매우 유능한 개인 비서야.
너는 아래와 같은 일을 할 수있어.

1. DB에서 팀, 멤버 정보 쿼리.
2. 캘린더 이벤트를 조정
3. 이메일을 전송 (DB에서 메일 주소 참조 필요)

사용자의 요청을 분석하여, 적절한 도구를 사용하고, 결과를 종합해야해.
요청이 여러가지 액션을 취해야 하면, 순서를 잘 짜서 각종 도구들을 여러번 호출해.
'''
supervisor_model = init_chat_model('gpt-4.1-mini')

supervisor_agent = create_agent(
    supervisor_model,
    tools=[schedule_event, manage_email, query_db],
    system_prompt=SUPERVISOR_PROMPT,
    checkpointer=InMemorySaver()
)

In [61]:
query = '아까 한거 오후 4시로 바꾸자'

config = {'configurable': {'thread_id': '12345d6'}}


for step in supervisor_agent.stream(
    {'messages': [{'role': 'user', 'content': query}]},
    config=config,
):
    for update in step.values():
        for message in update.get('messages', []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_2I14gKZCEWn6NdPIr709MxE3)
 Call ID: call_2I14gKZCEWn6NdPIr709MxE3
  Args:
    request: change the meeting with HR team tomorrow to 4pm
================================= Tool Message =================================
Name: schedule_event

The meeting with the HR team has been rescheduled to tomorrow at 4pm, from 16:00 to 17:00. If you need any further changes, please let me know!
================================== Ai Message ==================================
Tool Calls:
  manage_email (call_xBSeNkFRg8RBQn5WCsVfJuGO)
 Call ID: call_xBSeNkFRg8RBQn5WCsVfJuGO
  Args:
    request: Send an email to hr1@company.com, hr2@company.com, hr3@company.com, hr4@company.com, and hr5@company.com informing them that the meeting has been rescheduled to tomorrow at 4 PM and to confirm their attendance.
================================= Tool Message =================================
Name: man